# Лаб 4: RSA Public-Key Encryption and Signature Lab

*17B1NUM0662 Бямбаа Баярмандах*

*Мэдээлэл Технологи-4*

## Setup
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/byambaa0325/seed-security-labs/blob/master/notebooks/lab4-report.ipynb)

(Recommended) Тайланг Google Colab клоүд платформ дээр интерактив Jupyter Notebook-ээр бэлдсэн тул шууд ажлуулаад туршиж болно. Дээрх товчийг дараад Google Colab нөүтбүүк instance үүсгээд ажлуулж болно.

(Not Recommended) Хэрвээ өөрийн машин дээрээ ажлуулж байгаа бол [дараах линкээр](https://seedsecuritylabs.org/lab_env.html) ороод VM-ээ setup хийгээд нөүтбүүк дээр байгаа кодны нүднүүдийг хуулаад ажлуулж болно. 

Jupyter notebook ажиллуулах зааврыг [энэхүү линкээс](https://colab.research.google.com/notebooks/intro.ipynb) аваарай. Ерөнхийдөө, ажиллуулах нүдээ сонгоод `shift+enter` дараад байхад болно.

## Overview
RSA (Rivest–Shamir–Adleman) нь аюулгүй мэдээлэл дамжуулалтанд өргөн хэрэглэгддэг, анхдагч public-key cryptosystems юм. RSA нь хоёр маш том анхны тоо үүсгэн тэднээсээ public ба private key үүсгэж эдгээр түлхүүрээ нууцлал, тайлал, дижитал гарын үсгийн үүсгэл ба шалгалтанд ашигладаг.

Энэхүү лаб-аар RSA алгоритмийг туршиж, хэрэгжүүлж үзэх юм. Үүнд бид Jupyter Notebook дээр Python ашиглаж RSA-г хэрэгжүүлэх ба мөн дараах сэдвийг хамрана:
* Public-key cryptography
* The RSA algorithm and key generation
* Big number calculation
* Encryption and Decryption using RSA
* Digital signature
* X.509 certificate

**Readings and videos**. 
* Chapter 23 of the SEED Book, Computer & Internet Security: A Hands-on Approach, 2nd Edition,
by Wenliang Du. See details at https://www.handsonsecurity.net.


### Python 

Энэхүү лаб нь хуучнаар С дээр BIGNUM буюу том хэмжээний тоотой ажиллах санг ашиглан хийгддэг. Энэ нь C гэх мэт доод түвшний хийсвэрлэлттэй хэлнүүдэд тооны төрлүүд нь 32 битээр хязгаарлагддагтай холбоотой. RSA алгоритм нь 512 бит тоотой ажилладаг тул нэмэлт сангийн тусламжтайгаар ийм тоотой ажилладаг.

Харин Python-ий хувьд хэлийн int төрөл нь ямар ч хэмжээний тоотой ажиллах боломжтой байдаг. Энэ нь хэрэгжүүлэлтийн доод төвшиндөө динамикаар санах ойтой ажилладаг тул санах ойн хэмжээгээрээ ажиллаж болно гэсэн үг юм.

## Дасгал 1: Deriving the Private Key

`p,q,e` нь анхны тоо байг. `n` нь `n = p*q` байг. Бид `(e, n)`-ийг public key болгож ашиглах бол private key `d` хэд байхийг ол.


```
p = F7E75FDC469067FFDC4E847C51F452DF
q = E85CED54AF57E53E092113E62F436F4F
e = 0D88C3
```

Эдгээр тоо нь хэдий том болов ч аюулгүй байж чадхаар том биш юм. Практикт 512 битээр ашиглаж байж аюулгүй байдаг бол энд нь 128 бит юм.

In [ ]:
from math import gcd

p = int('F7E75FDC469067FFDC4E847C51F452DF', 16)
q = int('E85CED54AF57E53E092113E62F436F4F', 16)
e = int('0D88C3', 16)
v = (p-1)*(q-1)
n = p*q

Private key нь d^e%n = 1 байх d-ийн утга юм. Үүнийг олхын тулд модульийн эсрэгээр нь олох томьео ашиглана.

In [ ]:
def getModInverse(a, m):
    if gcd(a, m) != 1:
        return None
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m

    while v3 != 0:
        q = u3 // v3
        v1, v2, v3, u1, u2, u3 = (
            u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m
d = getModInverse(e,v)

Олдсон d:

In [ ]:
print(d)

24212225287904763939160097464943268930139828978795606022583874367720623008491


Шалгавал:


In [ ]:
(d*e)%v

1

Доорх нь ашиглах Public ба Private key юм.

In [ ]:
public_key = (e, n)
private_key = (d, n)

## Дасгал 2 ба 3: Encrypting and Decrypting a Message
Доорх түлхүүрүүдийг ашиглан RSA алгоритмаар мессежийг нууцлаад дараа нь буцааж тайлан ижил байгаа эсхийг шалгая.

In [ ]:
def encrypt(message, key):
  M = message.encode("utf-8").hex()
  M = int(M, 16)
  assert M < key[1]
  C = pow(M, key[0], key[1])
  return C

def decrypt(cipher, key):
  M = pow(cipher, key[0], key[1])
  M = hex(M)[2:]
  return bytes.fromhex(M).decode("utf-8")

In [ ]:
#Test Parameters
n = 'DCBFFE3E51F62E09CE7032E2677A78946A849DC4CDDE3A4D0CB81629242FB1A5'
e = '010001'
M = 'A top secret!'
d = '74D806F9F3A62BAE331FFE3F0A68AFE35B3D2E4794148AACBC26AA381CD7D30D'
public_key = (int(e,16), int(n,16))
private_key = (int(d,16), int(n,16))

Дараалаад нууцлаад тайлхад үндсэн мессеж ижил байгааг харж болно.

In [ ]:
decrypt(encrypt("Hi", public_key), private_key)

'Hi'

## Дасгал 4 ба 5: Signing a Message and Verifying a Signature

Нууцласан мессежэндээ өөрийн гарын үсгээ hex-ээр залгаад тухайн гарын үсгээ залгагдсан байгаа эсхийг шалгая. 


In [ ]:
def encrypt_hex(message, key, hex_signature = None):
  M = message.encode("utf-8").hex()
  #if hex_signature:
  #  M = M + hex_signature
  M = int(M, 16)
  #assert M < key[1]
  C = pow(M, key[0], key[1])
  if hex_signature:
    print(hex(C)[2:])
    C = hex(C)[2:] + hex_signature
  return C

def decrypt_hex(cipher, key, hex_signature = None):
  #M = pow(cipher, key[0], key[1])
  #M = hex(M)[2:]
  M = cipher
  if hex_signature:
    signature = M[-(len(hex_signature)):]
    M = M[:-len(hex_signature)]
    print(M)
    if signature != hex_signature:
      print("Signature invalid")
    else:
      print("Signature Valid")
  

Гарын үсэг баталгаажиж байгааг харж болно.

In [ ]:
decrypt_hex(encrypt_hex("I owe you $2000", public_key, hex_signature = "aa"), private_key, hex_signature = "aa")

16cdc2d574c9fdcf64a9e387f9ef69ab8bf9d6b839abcdbf617ef41ba12be37b
16cdc2d574c9fdcf64a9e387f9ef69ab8bf9d6b839abcdbf617ef41ba12be37b
Signature Valid


Тайлахгүйгээр нууцласан мессежийн ард залгагдсан эсхийг мөн шалгаж болно.

In [ ]:
M = 'Launch a missile.'
S = '643D6F34902D9C7EC90CB0B2BCA36C47FA37165C0005CAB026C0542CBDB6802F'
e = '010001'
n = 'AE1CD4DC432798D933779FBD46C6E1247F0CF1233595113AA51B450F18116115'
e = int(e, 16)
n = int(n, 16)
public_key = (e,n)
private_key = None
cipher = encrypt_hex(M, public_key, hex_signature = S)

cipher_hex = cipher

if cipher_hex[-(len(S)):] == S:
  print("Appended hex signature is valid")
else:
  print("Appended hex signature is invalid")


73f8c2374928dcfcf51b9fb4cf46520134b1ac6610f5af4854eef6ee03d47daa
Appended hex signature is valid


## Дасгал 6: Manually Verifying an X.509 Certificate

Энэ дасгалаар Х.509 certificate-ийг гараар баталгаажуулах болно. Ингэхдээ жинхэнэ certificate татаж аваад түүнийгээ шалгана.

Бидний ашиглах жишээ нь `example.org` юм. Эхлээд үүний сертификатыг татаж авая:


In [ ]:
! openssl s_client -connect www.example.org:443 -showcerts

CONNECTED(00000005)
depth=2 C = US, O = DigiCert Inc, OU = www.digicert.com, CN = DigiCert Global Root CA
verify return:1
depth=1 C = US, O = DigiCert Inc, CN = DigiCert SHA2 Secure Server CA
verify return:1
depth=0 C = US, ST = California, L = Los Angeles, O = Internet Corporation for Assigned Names and Numbers, OU = Technology, CN = www.example.org
verify return:1
---
Certificate chain
 0 s:C = US, ST = California, L = Los Angeles, O = Internet Corporation for Assigned Names and Numbers, OU = Technology, CN = www.example.org
   i:C = US, O = DigiCert Inc, CN = DigiCert SHA2 Secure Server CA
-----BEGIN CERTIFICATE-----
MIIHQDCCBiigAwIBAgIQD9B43Ujxor1NDyupa2A4/jANBgkqhkiG9w0BAQsFADBN
MQswCQYDVQQGEwJVUzEVMBMGA1UEChMMRGlnaUNlcnQgSW5jMScwJQYDVQQDEx5E
aWdpQ2VydCBTSEEyIFNlY3VyZSBTZXJ2ZXIgQ0EwHhcNMTgxMTI4MDAwMDAwWhcN
MjAxMjAyMTIwMDAwWjCBpTELMAkGA1UEBhMCVVMxEzARBgNVBAgTCkNhbGlmb3Ju
aWExFDASBgNVBAcTC0xvcyBBbmdlbGVzMTwwOgYDVQQKEzNJbnRlcm5ldCBDb3Jw
b3JhdGlvbiBmb3IgQXNzaWduZWQgTmFtZXMgYW5kIE51bWJ

Дээрх certificate-д хоёр сертификат байгаа. Үүнд subject ба issuer буюу сертификат авж буй сайтийн ба олгосон ямар нэгэн дундийн certificate authority-ийн өөрийн сертификат юм.

Эдгээрийг тус тусийн файлд бичхэд certificate бие буюу Begin Certificate гэсэн хэсгээс дуусах End Certificate хүртэл хуулж файл болгоно.

In [ ]:
%%writefile c0.pem
-----BEGIN CERTIFICATE-----
MIIHQDCCBiigAwIBAgIQD9B43Ujxor1NDyupa2A4/jANBgkqhkiG9w0BAQsFADBN
MQswCQYDVQQGEwJVUzEVMBMGA1UEChMMRGlnaUNlcnQgSW5jMScwJQYDVQQDEx5E
aWdpQ2VydCBTSEEyIFNlY3VyZSBTZXJ2ZXIgQ0EwHhcNMTgxMTI4MDAwMDAwWhcN
MjAxMjAyMTIwMDAwWjCBpTELMAkGA1UEBhMCVVMxEzARBgNVBAgTCkNhbGlmb3Ju
aWExFDASBgNVBAcTC0xvcyBBbmdlbGVzMTwwOgYDVQQKEzNJbnRlcm5ldCBDb3Jw
b3JhdGlvbiBmb3IgQXNzaWduZWQgTmFtZXMgYW5kIE51bWJlcnMxEzARBgNVBAsT
ClRlY2hub2xvZ3kxGDAWBgNVBAMTD3d3dy5leGFtcGxlLm9yZzCCASIwDQYJKoZI
hvcNAQEBBQADggEPADCCAQoCggEBANDwEnSgliByCGUZElpdStA6jGaPoCkrp9vV
rAzPpXGSFUIVsAeSdjF11yeOTVBqddF7U14nqu3rpGA68o5FGGtFM1yFEaogEv5g
rJ1MRY/d0w4+dw8JwoVlNMci+3QTuUKf9yH28JxEdG3J37Mfj2C3cREGkGNBnY80
eyRJRqzy8I0LSPTTkhr3okXuzOXXg38ugr1x3SgZWDNuEaE6oGpyYJIBWZ9jF3pJ
QnucP9vTBejMh374qvyd0QVQq3WxHrogy4nUbWw3gihMxT98wRD1oKVma1NTydvt
hcNtBfhkp8kO64/hxLHrLWgOFT/l4tz8IWQt7mkrBHjbd2XLVPkCAwEAAaOCA8Ew
ggO9MB8GA1UdIwQYMBaAFA+AYRyCMWHVLyjnjUY4tCzhxtniMB0GA1UdDgQWBBRm
mGIC4AmRp9njNvt2xrC/oW2nvjCBgQYDVR0RBHoweIIPd3d3LmV4YW1wbGUub3Jn
ggtleGFtcGxlLmNvbYILZXhhbXBsZS5lZHWCC2V4YW1wbGUubmV0ggtleGFtcGxl
Lm9yZ4IPd3d3LmV4YW1wbGUuY29tgg93d3cuZXhhbXBsZS5lZHWCD3d3dy5leGFt
cGxlLm5ldDAOBgNVHQ8BAf8EBAMCBaAwHQYDVR0lBBYwFAYIKwYBBQUHAwEGCCsG
AQUFBwMCMGsGA1UdHwRkMGIwL6AtoCuGKWh0dHA6Ly9jcmwzLmRpZ2ljZXJ0LmNv
bS9zc2NhLXNoYTItZzYuY3JsMC+gLaArhilodHRwOi8vY3JsNC5kaWdpY2VydC5j
b20vc3NjYS1zaGEyLWc2LmNybDBMBgNVHSAERTBDMDcGCWCGSAGG/WwBATAqMCgG
CCsGAQUFBwIBFhxodHRwczovL3d3dy5kaWdpY2VydC5jb20vQ1BTMAgGBmeBDAEC
AjB8BggrBgEFBQcBAQRwMG4wJAYIKwYBBQUHMAGGGGh0dHA6Ly9vY3NwLmRpZ2lj
ZXJ0LmNvbTBGBggrBgEFBQcwAoY6aHR0cDovL2NhY2VydHMuZGlnaWNlcnQuY29t
L0RpZ2lDZXJ0U0hBMlNlY3VyZVNlcnZlckNBLmNydDAMBgNVHRMBAf8EAjAAMIIB
fwYKKwYBBAHWeQIEAgSCAW8EggFrAWkAdwCkuQmQtBhYFIe7E6LMZ3AKPDWYBPkb
37jjd80OyA3cEAAAAWdcMZVGAAAEAwBIMEYCIQCEZIG3IR36Gkj1dq5L6EaGVycX
sHvpO7dKV0JsooTEbAIhALuTtf4wxGTkFkx8blhTV+7sf6pFT78ORo7+cP39jkJC
AHYAh3W/51l8+IxDmV+9827/Vo1HVjb/SrVgwbTq/16ggw8AAAFnXDGWFQAABAMA
RzBFAiBvqnfSHKeUwGMtLrOG3UGLQIoaL3+uZsGTX3MfSJNQEQIhANL5nUiGBR6g
l0QlCzzqzvorGXyB/yd7nttYttzo8EpOAHYAb1N2rDHwMRnYmQCkURX/dxUcEdkC
wQApBo2yCJo32RMAAAFnXDGWnAAABAMARzBFAiEA5Hn7Q4SOyqHkT+kDsHq7ku7z
RDuM7P4UDX2ft2Mpny0CIE13WtxJAUr0aASFYZ/XjSAMMfrB0/RxClvWVss9LHKM
MA0GCSqGSIb3DQEBCwUAA4IBAQBzcIXvQEGnakPVeJx7VUjmvGuZhrr7DQOLeP4R
8CmgDM1pFAvGBHiyzvCH1QGdxFl6cf7wbp7BoLCRLR/qPVXFMwUMzcE1GLBqaGZM
v1Yh2lvZSLmMNSGRXdx113pGLCInpm/TOhfrvr0TxRImc8BdozWJavsn1N2qdHQu
N+UBO6bQMLCD0KHEdSGFsuX6ZwAworxTg02/1qiDu7zW7RyzHvFYA4IAjpzvkPIa
X6KjBtpdvp/aXabmL95YgBjT8WJ7pqOfrqhpcmOBZa6Cg6O1l4qbIFH/Gj9hQB5I
0Gs4+eH6F9h3SojmPTYkT+8KuZ9w84Mn+M8qBXUQoYoKgIjN
-----END CERTIFICATE-----

In [ ]:
%%writefile c1.pem
-----BEGIN CERTIFICATE-----
MIIElDCCA3ygAwIBAgIQAf2j627KdciIQ4tyS8+8kTANBgkqhkiG9w0BAQsFADBh
MQswCQYDVQQGEwJVUzEVMBMGA1UEChMMRGlnaUNlcnQgSW5jMRkwFwYDVQQLExB3
d3cuZGlnaWNlcnQuY29tMSAwHgYDVQQDExdEaWdpQ2VydCBHbG9iYWwgUm9vdCBD
QTAeFw0xMzAzMDgxMjAwMDBaFw0yMzAzMDgxMjAwMDBaME0xCzAJBgNVBAYTAlVT
MRUwEwYDVQQKEwxEaWdpQ2VydCBJbmMxJzAlBgNVBAMTHkRpZ2lDZXJ0IFNIQTIg
U2VjdXJlIFNlcnZlciBDQTCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEB
ANyuWJBNwcQwFZA1W248ghX1LFy949v/cUP6ZCWA1O4Yok3wZtAKc24RmDYXZK83
nf36QYSvx6+M/hpzTc8zl5CilodTgyu5pnVILR1WN3vaMTIa16yrBvSqXUu3R0bd
KpPDkC55gIDvEwRqFDu1m5K+wgdlTvza/P96rtxcflUxDOg5B6TXvi/TC2rSsd9f
/ld0Uzs1gN2ujkSYs58O09rg1/RrKatEp0tYhG2SS4HD2nOLEpdIkARFdRrdNzGX
kujNVA075ME/OV4uuPNcfhCOhkEAjUVmR7ChZc6gqikJTvOX6+guqw9ypzAO+sf0
/RR3w6RbKFfCs/mC/bdFWJsCAwEAAaOCAVowggFWMBIGA1UdEwEB/wQIMAYBAf8C
AQAwDgYDVR0PAQH/BAQDAgGGMDQGCCsGAQUFBwEBBCgwJjAkBggrBgEFBQcwAYYY
aHR0cDovL29jc3AuZGlnaWNlcnQuY29tMHsGA1UdHwR0MHIwN6A1oDOGMWh0dHA6
Ly9jcmwzLmRpZ2ljZXJ0LmNvbS9EaWdpQ2VydEdsb2JhbFJvb3RDQS5jcmwwN6A1
oDOGMWh0dHA6Ly9jcmw0LmRpZ2ljZXJ0LmNvbS9EaWdpQ2VydEdsb2JhbFJvb3RD
QS5jcmwwPQYDVR0gBDYwNDAyBgRVHSAAMCowKAYIKwYBBQUHAgEWHGh0dHBzOi8v
d3d3LmRpZ2ljZXJ0LmNvbS9DUFMwHQYDVR0OBBYEFA+AYRyCMWHVLyjnjUY4tCzh
xtniMB8GA1UdIwQYMBaAFAPeUDVW0Uy7ZvCj4hsbw5eyPdFVMA0GCSqGSIb3DQEB
CwUAA4IBAQAjPt9L0jFCpbZ+QlwaRMxp0Wi0XUvgBCFsS+JtzLHgl4+mUwnNqipl
5TlPHoOlblyYoiQm5vuh7ZPHLgLGTUq/sELfeNqzqPlt/yGFUzZgTHbO7Djc1lGA
8MXW5dRNJ2Srm8c+cftIl7gzbckTB+6WohsYFfZcTEDts8Ls/3HB40f/1LkAtDdC
2iDJ6m6K7hQGrn2iWZiIqBtvLfTyyRRfJs8sjX7tN8Cp1Tm5gr8ZDOo0rwAhaPit
c+LJMto4JQtV05od8GiG7S5BNO98pVAdvzr508EIDObtHopYJeS4d60tbvVS3bR0
j6tJLp07kzQoH3jOlOrHvdPJbRzeXDLz
-----END CERTIFICATE-----

Олгогчийн сертификатаас public key-ийн `n`-ийн утгыг авцгаая:

In [ ]:
! openssl x509 -in c1.pem -noout -modulus

Олгогчийн сертификатаас public key-ийн `e`-ийн утгыг авцгаая. Ингэхдээ хэвлэсэн утгаас Exponent: гэж тэмдэглэсэн тоо нь e юм.

In [ ]:
! openssl x509 -in c1.pem -text -noout

Сайтийн сертификатийн signature-ийг мөн авая. Үүнд мөн адил хэвлэсэн утгаас олж хуулж авна.

In [ ]:
! openssl x509 -in c0.pem -text -noout

Үүнийг signature гэх файлд бичэе

In [ ]:
%%writefile signature
73:70:85:ef:40:41:a7:6a:43:d5:78:9c:7b:55:48:e6:bc:6b:
99:86:ba:fb:0d:03:8b:78:fe:11:f0:29:a0:0c:cd:69:14:0b:
c6:04:78:b2:ce:f0:87:d5:01:9d:c4:59:7a:71:fe:f0:6e:9e:
c1:a0:b0:91:2d:1f:ea:3d:55:c5:33:05:0c:cd:c1:35:18:b0:
6a:68:66:4c:bf:56:21:da:5b:d9:48:b9:8c:35:21:91:5d:dc:
75:d7:7a:46:2c:22:27:a6:6f:d3:3a:17:eb:be:bd:13:c5:12:
26:73:c0:5d:a3:35:89:6a:fb:27:d4:dd:aa:74:74:2e:37:e5:
01:3b:a6:d0:30:b0:83:d0:a1:c4:75:21:85:b2:e5:fa:67:00:
30:a2:bc:53:83:4d:bf:d6:a8:83:bb:bc:d6:ed:1c:b3:1e:f1:
58:03:82:00:8e:9c:ef:90:f2:1a:5f:a2:a3:06:da:5d:be:9f:
da:5d:a6:e6:2f:de:58:80:18:d3:f1:62:7b:a6:a3:9f:ae:a8:
69:72:63:81:65:ae:82:83:a3:b5:97:8a:9b:20:51:ff:1a:3f:
61:40:1e:48:d0:6b:38:f9:e1:fa:17:d8:77:4a:88:e6:3d:36:
24:4f:ef:0a:b9:9f:70:f3:83:27:f8:cf:2a:05:75:10:a1:8a:
0a:80:88:cd

Үүнээс илүү зай ба тэмдэгтүүдийг хасаж hex хэлбэрт шилжүүлвэл:

In [ ]:
! cat signature | tr -d ’[:space:]:’

In [ ]:
%%writefile signature
737085ef4041a76a43d5789c7b5548e6bc6b9986bafb0d038b78fe11f029a00ccd69140bc60478b2cef087d5019dc4597a71fef06e9ec1a0b0912d1fea3d55c533050ccdc13518b06a68664cbf5621da5bd948b98c3521915ddc75d77a462c2227a66fd33a17ebbebd13c5122673c05da335896afb27d4ddaa74742e37e5013ba6d030b083d0a1c4752185b2e5fa670030a2bc53834dbfd6a883bbbcd6ed1cb31ef1580382008e9cef90f21a5fa2a306da5dbe9fda5da6e62fde588018d3f1627ba6a39faea86972638165ae8283a3b5978a9b2051ff1a3f61401e48d06b38f9e1fa17d8774a88e63d36244fef0ab99f70f38327f8cf2a057510a18a0a8088cd

In [ ]:
! openssl asn1parse -i -in c0.pem

Сертификатийн биеийн хэсгийн аваад

In [ ]:
! openssl asn1parse -i -in c0.pem -strparse 4 -out c0_body.bin -noout

Хашийг нь бодвол:

In [ ]:
! sha256sum c0_body.bin

Хэрэгтэй зүйлүүд буюу CA-ийн public key, signature, мөн сертификатийн хашуудаа гаргаж авлаа. Одоо үүнийг шалгахийн тулд signature-аа public key-ээрээ тайлаад сертификатийн хаштай ижил байгааг шалгахад зөв эсхийг мэдэж болно.

In [ ]:
n = 'DCAE58904DC1C4301590355B6E3C8215F52C5CBDE3DBFF7143FA642580D4EE18A24DF066D0\
0A736E1198361764AF379DFDFA4184AFC7AF8CFE1A734DCF339790A2968753832BB9A675482D1D5\
6377BDA31321AD7ACAB06F4AA5D4BB74746DD2A93C3902E798080EF13046A143BB59B92BEC20765\
4EFCDAFCFF7AAEDC5C7E55310CE83907A4D7BE2FD30B6AD2B1DF5FFE5774533B3580DDAE8E4498B\
39F0ED3DAE0D7F46B29AB44A74B58846D924B81C3DA738B129748900445751ADD37319792E8CD54\
0D3BE4C13F395E2EB8F35C7E108E8641008D456647B0A165CEA0AA29094EF397EBE82EAB0F72A73\
00EFAC7F4FD1477C3A45B2857C2B3F982FDB745589B'
signature = '737085ef4041a76a43d5789c7b5548e6bc6b9986bafb0d038b78fe11f029a00ccd\
69140bc60478b2cef087d5019dc4597a71fef06e9ec1a0b0912d1fea3d55c533050ccdc13518b06\
a68664cbf5621da5bd948b98c3521915ddc75d77a462c2227a66fd33a17ebbebd13c5122673c05d\
a335896afb27d4ddaa74742e37e5013ba6d030b083d0a1c4752185b2e5fa670030a2bc53834dbfd\
6a883bbbcd6ed1cb31ef1580382008e9cef90f21a5fa2a306da5dbe9fda5da6e62fde588018d3f1\
627ba6a39faea86972638165ae8283a3b5978a9b2051ff1a3f61401e48d06b38f9e1fa17d8774a8\
8e63d36244fef0ab99f70f38327f8cf2a057510a18a0a8088cd'
body_hash = '2c2a46bf245dab54ddb47298621e9629309f0e2c90c4d80d535c7d4e8ab07d29'
e = 65537
n = int(n,16)
public_key = (e, n)
s = int(signature, 16)
decoded_signature = pow(s,e,n)
decoded_signature_hex = hex(decoded_signature)[2:]

print(decoded_signature_hex[-len(body_hash):])
print(body_hash)
if decoded_signature_hex[-len(body_hash):] == body_hash:
  print("Signature valid")
else:
  print("Signature invalid")
